In [9]:
# 队列
class Queue:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def enqueue(self, item):  # 入队
        self.items.insert(0, item)

    def dequeue(self):  # 出队
        return self.items.pop()

    def size(self):
        return len(self.items)


# 解决8数码问题
class State():
    def __init__(self, list):
        # list为当前数码分布，列表表示
        self.value = list
        self.dist = 0  # 到达当前状态的步数
        self.color = 'white'  # white表示未访问，gray表示已访问,black表示已探索
        self.pred = None  # 前驱节点,用于记录路径
    
    def getValue(self):
        return self.value
    
    def setValue(self, list):
        self.value = list
        
    def getDist(self):
        return self.dist
    
    def setDist(self, dist):
        self.dist = dist
        
    def getColor(self):
        return self.color
    
    def setColor(self, color):
        self.color = color
        
    def getPred(self):
        return self.pred
    
    def setPred(self, pred):
        self.pred = pred
        

# 寻找所有的邻居节点，上下左右移动
def findNeighbors(state):
    # state为当前状态，State类
    current_state = state.getValue()  # 当前状态
    zero_index = current_state.index(0)   # 找到0的位置
    list_location = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # 上下左右移动
    for location in list_location:
        new_index = zero_index + location[0] * 3 + location[1]
        if new_index < 0 or new_index > 8:
            continue
        new_state = current_state.copy()
        new_state[zero_index] = new_state[new_index]
        new_state[new_index] = 0
        new_state = State(new_state)
        yield new_state

# bfs寻找最短路径
def bfsCode8(start, end):
    # start为初始状态，end为目标状态,均为元组
    current_state = State(list(start))  # 将当前状态设置为初始状态
    end_state = State(list(end))  # 目标状态
    vertQueue = Queue()  # bfs使用队列，先进先出
    vertQueue.enqueue(current_state)  # 入队
    dist_min = 10000  # 记录为到达目标状态的最小步数
    set_visited = set()  # 记录访问过的顶点
    
    while vertQueue.size() > 0:
        current_state = vertQueue.dequeue()  # 出队
        for state in findNeighbors(current_state):
            # 跳过已经访问的顶点
            if str(state.getValue()) in set_visited:
                continue
            
            # 更新顶点
            state.setColor('Grey')  # 设定颜色为灰色，表示已访问
            state.setDist(current_state.getDist() + 1)  # 步数加1
            state.setPred(current_state)
            
            # 入队
            vertQueue.enqueue(state)
            
            # 更新集合
            set_visited.add(str(state.getValue()))
            
            # 判断是否到达目标状态
            if state.getValue() == end_state.getValue():
                dist_min = min(dist_min, state.getDist())  # 更新最小步数
        current_state.setColor('Black')  # 设定颜色为黑色，表示已探索
    return dist_min


# 测试            
if __name__ == '__main__':
    start = (2, 8, 3, 1, 6, 4, 7, 0, 5)
    end = (1, 2, 3, 8, 0, 4, 7, 6, 5)
    print(bfsCode8(start, end))

5


In [23]:
# DFS解决八皇后问题

# 栈
# 此时认为列表的尾部为栈的顶部
class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def peek(self):
        return self.items[len(self.items)-1]

    def size(self):
        return len(self.items)


class State():
    def __init__(self, list):
        # list为当前数码分布，列表表示
        self.value = list
        self.pred = None  # 前驱节点,用于记录路径
    
    def getValue(self):
        return self.value
    
    def setValue(self, list):
        self.value = list
        
    def getPred(self):
        return self.pred
    
    def setPred(self, pred):
        self.pred = pred


def dfsQueen():
    # 利用DFS解决八皇后问题
    stack = Stack()  # 使用栈
    nums_solve = 0  # 记录解的个数
    for index in range(8):
        stack.push(State([index, 0]))  # 将第一个皇后放入栈中
        while stack.size() > 0:
            current_state = stack.pop() # 出栈

            if current_state.getValue()[1] == 7:  # 判断是否到达最后一行
                nums_solve += 1
                continue
            
            neighbors = list(findNeighbors(current_state))  # 寻找邻居节点
            if len(neighbors) == 0:
                continue
            for neighbor in neighbors:
                stack.push(neighbor)
    return nums_solve


# 判断是否冲突
def isConflict(state, new_state):
    # 判断是否冲突
    # state为当前状态，new_state为新状态
    current_index = state.getValue()  # 当前状态
    new_index = new_state.getValue()  # 新状态
    x1, y1 = current_index
    x2, y2 = new_index
    if x1 == x2 or y1 == y2 or abs(x1-x2) == abs(y1-y2):
        return True
    return False

# 检查所有节点是否冲突
def checkConflict(state, new_state):
    # state为当前状态，State类
    if isConflict(state, new_state):
        return True
    pred = state.getPred()  # 前驱节点
    while pred:
        if isConflict(pred, new_state):
            return True
        pred = pred.getPred()
    return False

# 寻找所有的邻居节点
def findNeighbors(state):
    # state为当前状态，State类
    current_index = state.getValue()  # 当前状态
    y = current_index[1]  # 当前行数
    for x in range(8):
        new_state = State([x, y+1])  # 新状态
        new_state.setPred(state)
        if checkConflict(state, new_state):
            continue
        yield new_state


# 测试
if __name__ == '__main__':
    print(dfsQueen())

92


In [4]:
# 实现拓扑排序
class Vertex():
    def __init__(self, key):
        # 顶点的id
        self.id = key
        # 顶点的连接,以字典的形式存储，key是顶点对象，value是权重
        self.connectedTo = {}
        
        # 针对广度优先搜索的辅助变量
        self.color = 'white'  # 顶点的颜色，白色表示未访问，灰色表示访问过但未探索，黑色表示访问过且探索过
        self.dist = 0  # 顶点的距离
        self.pred = None  # 顶点的前驱
        
        # 针对深度优先搜索的辅助变量
        self.disc = 0
        self.fin = 0
    
    def addNeighbor(self, nbr, weight=0):
        # 添加邻接顶点
        self.connectedTo[nbr] = weight
        
    def __str__(self):
        # 返回顶点的连接信息
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])
    
    def getConnections(self):
        # 返回所有连接的顶点
        return self.connectedTo.keys()
    
    def getId(self):
        # 返回顶点的id
        return self.id
    
    def getWeight(self, nbr):
        # 返回顶点的权重
        return self.connectedTo[nbr]
    
    # 针对广度优先搜索的辅助函数
    # 针对广度优先搜索的辅助函数
    # 针对广度优先搜索的辅助函数
    def setColor(self, color):
        self.color = color
        
    def getColor(self):
        return self.color
    
    def setDistance(self, d):
        self.dist = d
        
    def getDistance(self):
        return self.dist
    
    def setPred(self, p):
        self.pred = p
        
    def getPred(self):
        return self.pred
    
    # 针对深度优先搜索的辅助函数
    # 针对深度优先搜索的辅助函数
    # 针对深度优先搜索的辅助函数
    def setDiscovery(self, dtime):
        self.disc = dtime
        
    def getDiscovery(self):
        return self.disc
    
    def setFinish(self, ftime):
        self.fin = ftime
        
    def getFinish(self):
        return self.fin


# 实现Graph类
class Graph():
    def __init__(self):
        # 图的顶点列表
        self.vertList = {}
        # 顶点的个数
        self.numVertices = 0
    
    def addVertex(self, key):
        # 添加顶点
        self.numVertices += 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex
    
    def getVertex(self, n):
        # 返回顶点
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None
        
    def __contains__(self, n):
        # 判断顶点是否在图中
        return n in self.vertList
    
    def addEdge(self, f, t, cost=0):
        # 添加边
        # f是起点，t是终点，cost是权重
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], cost)
        
    def getVertices(self):
        # 返回所有顶点
        return self.vertList.keys()
    
    def __iter__(self):
        # 迭代器
        # 返回所有顶点
        # 形式为每一个顶点相连的所以顶点
        return iter(self.vertList.values())
    

# 生成一个DAG图
def buildGraph():
    g = Graph()
    for i in range(6):
        g.addVertex(i)
    g.addEdge(0, 1, 5)
    g.addEdge(0, 2, 3)
    g.addEdge(1, 3, 6)
    g.addEdge(1, 4, 4)
    g.addEdge(2, 1, 2)
    g.addEdge(2, 4, 7)
    g.addEdge(3, 4, -1)
    g.addEdge(3, 5, 1)
    g.addEdge(4, 5, -2)
    return g

# 队列
class Queue:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def enqueue(self, item):  # 入队
        self.items.insert(0, item)

    def dequeue(self):  # 出队
        return self.items.pop()

    def size(self):
        return len(self.items)

# BFS拓扑排序
def bfsTopo(g):
    # g为图
    # 生成入度字典
    in_degree = {}
    for vertex in g:
        in_degree[vertex] = 0
    for vertex in g:
        for neighbor in vertex.getConnections():
            in_degree[neighbor] += 1
    # 入度为0的顶点入队
    queue = Queue()
    for vertex in in_degree:
        if in_degree[vertex] == 0:
            queue.enqueue(vertex)
    # 拓扑排序
    while queue.size() > 0:
        current_vertex = queue.dequeue()
        print(current_vertex.getId())
        for neighbor in current_vertex.getConnections():
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.enqueue(neighbor)
                
# 测试
if __name__ == '__main__':
    g = buildGraph()
    bfsTopo(g)
    

# DFS拓扑排序
def dfsTopo(g):
    # g为图
    # 时间戳
    time = 0
    # 深度优先搜索
    def dfsVisit(vertex):
        nonlocal time
        time += 1
        vertex.setDiscovery(time)
        vertex.setColor('gray')
        for neighbor in vertex.getConnections():
            if neighbor.getColor() == 'white':
                neighbor.setPred(vertex)
                dfsVisit(neighbor)
        vertex.setColor('black')
        time += 1
        vertex.setFinish(time)
        print(vertex.getId())
    # 初始化
    for vertex in g:
        vertex.setColor('white')
        vertex.setPred(None)
    # 深度优先搜索
    for vertex in g:
        if vertex.getColor() == 'white':
            dfsVisit(vertex)
            
# 测试
if __name__ == '__main__':
    g = buildGraph()
    dfsTopo(g)

0
2
1
3
4
5
5
4
3
1
2
0
